In [1]:
import pandas as pd

In [2]:
data_path = './corona_scotland.xlsx'

# Cases and Death Data

In [3]:
df = pd.read_excel(data_path, 
                   dtype={'Total Tests': 'Int32',
                          'Total Positive Cases': 'Int32',
                          'Total Deaths': 'Int32'}
                   ).ffill()
df = df[df['Health Board'] != 'Golden Jubilee National Hospital']

In [4]:
df

,Date,Health Board,Number of Positive cases,Total Tests,Total Positive Cases,Total Deaths
0,2020-03-06,Tayside,1,1514,11,0
1,2020-03-06,Ayrshire & Arran,1,1514,11,0
2,2020-03-06,Forth Valley,2,1514,11,0
3,2020-03-06,Greater Glasgow and Clyde,1,1514,11,0
4,2020-03-06,Grampian,3,1514,11,0
...,...,...,...,...,...,...
1036,2020-05-21,Lothian,2611,95173,14856,2221
1037,2020-05-21,Orkney,7,95173,14856,2221
1038,2020-05-21,Shetland,54,95173,14856,2221
1039,2020-05-21,Tayside,1632,95173,14856,2221


## clean data

In [5]:
df['Health Board'].value_counts()

Lothian                          77
Fife                             77
Tayside                          77
Grampian                         77
Greater Glasgow and Clyde        77
Forth Valley                     77
Lanarkshire                      76
Shetland                         74
Borders                          72
Highland                         69
Dumfries and Galloway            67
Ayrshire and Arran               54
Eileanan Siar (Western Isles)    51
Orkney                           51
Ayrshire and Arran               20
Ayrshire & Arran                  3
Name: Health Board, dtype: int64

In [6]:
# clean up some names
df.loc[df['Health Board'] == 'Ayrshire\xa0and Arran', 'Health Board'] = 'Ayrshire and Arran' 
df.loc[df['Health Board'] == 'Ayrshire & Arran', 'Health Board'] = 'Ayrshire and Arran' 

In [7]:
df['Health Board'].value_counts()

Lothian                          77
Fife                             77
Tayside                          77
Grampian                         77
Ayrshire and Arran               77
Forth Valley                     77
Greater Glasgow and Clyde        77
Lanarkshire                      76
Shetland                         74
Borders                          72
Highland                         69
Dumfries and Galloway            67
Eileanan Siar (Western Isles)    51
Orkney                           51
Name: Health Board, dtype: int64

## table of number cases per region over time

In [8]:
df_cases = df[['Date', 'Health Board', 'Number of Positive cases']]

In [9]:
df_table = pd.pivot_table(df_cases, 
                          columns=['Health Board'], 
                          values=['Number of Positive cases'], 
                          index=['Date'],
                          fill_value=0,
                         ).astype('Int32')

df_table.columns = df_table.columns.droplevel(0)

In [10]:
df_table['Scotland (Total)'] = df_table.sum(axis=1)

In [11]:
df_table.to_pickle('corona_scotland_regions_ts.pk')

## table of national cases, tests, and deaths over time

In [12]:
df_national = (df.drop(columns=['Health Board', 'Number of Positive cases'])
                 .groupby('Date').first()
              )

In [13]:
df_national.index

DatetimeIndex(['2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09',
               '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
               '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17',
               '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21',
               '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25',
               '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29',
               '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02',
               '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06',
               '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10',
               '2020-04-11', '2020-04-12', '2020-04-13', '2020-04-14',
               '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-18',
               '2020-04-19', '2020-04-20', '2020-04-21', '2020-04-22',
               '2020-04-23', '2020-04-24', '2020-04-25', '2020-04-26',
               '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30',
      

In [14]:
df_national.to_pickle('corona_scotland_total_stats.pk')

## Scotland Population Data

In [15]:
region_pops = {
    'Ayrshire and Arran': 369670,
    'Borders': 115270, 
    'Dumfries and Galloway': 148790,
    'Fife': 371910,
    'Forth Valley': 306070,
    'Grampian': 584550,
    'Greater Glasgow and Clyde': 1196335,
    'Highland': 321800,
    'Lanarkshire': 563185,
    'Lothian': 897770,
    'Orkney': 22190,
    'Shetland': 22990,
    'Tayside': 416080,
    'Eileanan Siar (Western Isles)': 26500,
    #'Scotland (Total)': 5438000,
}

In [16]:
df_pop = pd.DataFrame(region_pops.items(), columns=['Region', 'Pop']).set_index('Region').sort_index()

In [17]:
df_pop.loc['Scotland (Total)'] = 5438000

In [18]:
df_pop

,Pop
Region,
Ayrshire and Arran,369670
Borders,115270
Dumfries and Galloway,148790
Eileanan Siar (Western Isles),26500
Fife,371910
Forth Valley,306070
Grampian,584550
Greater Glasgow and Clyde,1196335
Highland,321800


In [19]:
df_pop[:14].sum()
## hmm missing few thousands??

Pop    5363110
dtype: int64

In [20]:
df_pop.to_pickle('scotland_population.pk')

# Hospital Data

In [21]:
from datetime import datetime

In [22]:
df = pd.read_excel(data_path, 
                   usecols=['date', 'health_board', 
                            'num_hospital', 'num_icu'],
                   dtype={
                       'date': datetime,
                       'health_board': 'str',
                       'num_hospital': 'Int32',
                       'num_icu': 'Int32'},
                   na_values={'num_hospital': '*',
                              'num_icu': '*'},
                   sheet_name=1,
                   )
df['date'].ffill(inplace=True)
df.fillna(0, inplace=True)

In [23]:
df

,date,health_board,num_hospital,num_icu
0,2020-04-08,Ayrshire and Arran,91,16
1,2020-04-08,Borders,56,9
2,2020-04-08,Dumfries and Galloway,44,6
3,2020-04-08,Fife,127,13
4,2020-04-08,Forth Valley,107,9
...,...,...,...,...
654,2020-05-21,Orkney,0,0
655,2020-05-21,Shetland,0,0
656,2020-05-21,Tayside,36,0
657,2020-05-21,Eileanan Siar (Western Isles),0,0


In [24]:
df_hos = df[['date', 'health_board', 'num_hospital']]
df_icu = df[['date', 'health_board', 'num_icu']]

In [25]:
df_table = pd.pivot_table(df_hos, 
                          columns=['health_board'], 
                          values=['num_hospital'], 
                          index=['date'],
                          fill_value=0,
                         ).astype('int32')

df_table.columns = df_table.columns.droplevel(0)
df_table['Scotland (Total)'] = df_table.sum(axis=1)

In [26]:
df_table.tail()

health_board,Ayrshire and Arran,Borders,Dumfries and Galloway,Eileanan Siar (Western Isles),Fife,Forth Valley,Golden Jubilee National Hospital,Grampian,Greater Glasgow and Clyde,Highland,Lanarkshire,Lothian,Orkney,Shetland,Tayside,Scotland (Total)
date,,,,,,,,,,,,,,,,
2020-05-17,80,40,15,0,97,62,0,117,432,36,135,244,0,0,43,1301
2020-05-18,89,44,10,0,103,56,0,149,434,28,166,282,0,0,56,1417
2020-05-19,95,35,11,0,108,66,0,139,428,62,172,274,0,0,43,1433
2020-05-20,98,41,22,0,103,62,0,113,412,46,191,307,0,0,37,1432
2020-05-21,70,33,12,0,91,55,0,104,400,41,148,322,0,0,36,1312


In [27]:
df_table.to_pickle('corona_scotland_regions_hospital_ts.pk')

In [28]:
df_table = pd.pivot_table(df_icu, 
                          columns=['health_board'], 
                          values=['num_icu'], 
                          index=['date'],
                          fill_value=0,
                         ).astype('int32')

df_table.columns = df_table.columns.droplevel(0)
df_table['Scotland (Total)'] = df_table.sum(axis=1)

In [29]:
df_table.tail()

health_board,Ayrshire and Arran,Borders,Dumfries and Galloway,Eileanan Siar (Western Isles),Fife,Forth Valley,Golden Jubilee National Hospital,Grampian,Greater Glasgow and Clyde,Highland,Lanarkshire,Lothian,Orkney,Shetland,Tayside,Scotland (Total)
date,,,,,,,,,,,,,,,,
2020-05-17,0,0,0,0,0,0,0,5,20,0,11,7,0,0,0,43
2020-05-18,5,0,0,0,0,0,0,7,20,0,10,10,0,0,0,52
2020-05-19,0,0,0,0,0,0,0,6,20,0,0,0,0,0,0,26
2020-05-20,0,0,0,0,0,0,0,6,18,0,8,8,0,0,0,40
2020-05-21,0,0,0,0,0,0,0,6,16,0,0,12,0,0,0,34


In [30]:
df_table.to_pickle('corona_scotland_regions_icu_ts.pk')